In [24]:
import datetime
import sys

from bs4 import BeautifulSoup
import requests
from urllib.request import urlopen

import pandas as pd
from pandas_datareader import data
from itertools import islice

from scipy.signal import savgol_filter
import statsmodels
import statsmodels.api as sm
import scipy.stats as stats
import scipy.optimize as optimize
import numpy as np
#from mpl_finance import candlestick_ohlc
import matplotlib.pyplot as plt
#For the usual plots!
#matplotlib inline
#Makes graphs in jupyter interactive!
%matplotlib widget

tickers = ['SPY', 'AAPL', 'MSFT', 'PYPL', 'ISRG', 'GM', 'KODK', 'INTC', 'PANW']

#Removing warnings
import warnings
warnings.filterwarnings('ignore')

In [25]:
plt.close('all') 

In [26]:
def stock_data(startd, endd, tickers=tickers):
    df = data.DataReader(tickers, 'yahoo', startd, endd).reset_index()
#df.set_index('Date', inplace=True)            <-- weggehaald zodat polyfit en backtest "i-1" werken
#uitrekenen van de daily return (cert)
    for ticker in tickers:
        df[('Return', ticker)] = df[('Close', ticker)].shift(1) - df[('Close', ticker)] 
    for ticker in tickers:
        df[('Percent_return', ticker)] = df[('Close', ticker)].pct_change()
    return df

def ehlers_simple_decycle(df, src, hpLength, t, name):
    #hp = highpass
    upper = 0.5
    lower = 0.5
    alphaArg = 2*np.pi / (hpLength*np.sqrt(2))
    alpha = 'Alpha_' + name
    hp = 'Hp_' + name
    decycler = 'Decycler_' + name
    df[(alpha, t)] = 0        
    df[(hp, t)] = 0   
    df[(decycler, t)] = 0  
    for i in range(125, len(df)):
        #Hp filter
        if np.cos(alphaArg) != 0:
            df.loc[i, (alpha, t)] = (np.cos(alphaArg)+np.sin(alphaArg)-1) / np.cos(alphaArg)
        else:
            df.loc[i, (alpha, t)] = df.loc[i-1, (alpha, t)]
        #Decycling::: src = the input!
        df.loc[i, (hp, t)] = np.square(1-(df.loc[i, (alpha, t)]/2)) * (src[i]-2 * src[i-1]+src[i-2]) + 2 * (1-df.loc[i, (alpha, t)]) * df.loc[i-1, (hp, t)] - np.square(1-df.loc[i, (alpha, t)]) * df.loc[i-2, (hp, t)]
        df.loc[i, (decycler, t)] = src[i] - df.loc[i, (hp, t)]
    return df
    
def bolinger_bands(df, close, length, multiple):
    for i in range(len(df)):
        mean_bb = close.rolling(length).mean()
        std  = close.rolling(length).std()
        upper_bb = mean_bb + (std*multiple)
        lower_bb = mean_bb - (std*multiple)
        return mean_bb, upper_bb, lower_bb   
    
#Functions Average True Range#
def wwma(values, n):
    return values.ewm(alpha=1/n, adjust=False).mean()

def atr(data, t , n, multiplier):
    high = data[('High', t)]
    low = data[('Low', t)]
    close = data[('Close', t)]
    data[('Tr0', t)] = abs(high - low)
    data[('Tr1', t)] = abs(high - close.shift())
    data[('Tr2', t)] = abs(low - close.shift())
    Tr = data[[('Tr0', t), ('Tr1', t), ('Tr2', t)]].max(axis=1)
    Atr = wwma(Tr, n)
    data[('Atr', t)] = Atr
#De Stoploss 2xATR implementatie:
    Stoploss = data[('Close', t)] - (data[('Atr', t)] * multiplier)
    return Stoploss 

In [27]:
################                    ALLE INPUTS (Dataframes)

start_time = datetime.datetime.now().time()
print(start_time)

tickers = ['SPY', 'AAPL', 'MSFT', 'PYPL', 'ISRG', 'GM', 'INTC', 'PANW']
tickers = ['SPY']
startd = datetime.datetime(2016, 1, 1) 
#endd_1 = datetime.datetime(2019, 1, 1)
endd_2 = datetime.datetime(2020, 1, 1)
df = data.DataReader(tickers, 'yahoo', startd, endd_2).reset_index()
for t in tickers:
    df[('Percent_return', t)] = df[('Close', t)].pct_change()
    df[('Percent_return', 'Total')] = df[('Close', t)].pct_change()
print(df.index.max())

###Needed for SELLING
#BB with 0.75 std 20ma
for t in tickers:
    df[('Mean_bb', t)], df[('Upper_bb', t)], df[('Lower_bb', t)] = bolinger_bands(df, df[('Close', t)], 20, 0.75) 

#ATR #1  
for t in tickers:
    df[('Stoploss_atr3', t)] = atr(df, t, 1, 3)
    
#ATR #2  
for t in tickers:
    df[('Stoploss_atr1', t)] = atr(df, t, 1, 1)

#Decylcer 75
for t in tickers:
    df = ehlers_simple_decycle(df, df[('Close', t)], 75, t, 'c75')

    
    
###Needed for BUYING
#Decylcer 25
for t in tickers:
    df = ehlers_simple_decycle(df, df[('Open', t)], 25, t, 'o25')

#Decylcer 75
for t in tickers:
    df = ehlers_simple_decycle(df, df[('Open', t)], 75, t, 'o75')

#Sma10
for t in tickers:
    df[('SMA10', t)] = df[('Close', t)].rolling(10).mean()
    
#Sma20
for t in tickers:
    df[('SMA20', t)] = df[('Close', t)].rolling(20).mean()
    
time = datetime.datetime.now().time()
print(time)
print(df.index.max())

03:32:35.960194
1005
03:32:46.998379
1005


In [28]:
############################              ALLE SIGNALS!!!!
#maak eerst alles even los van elkaar!

for t in tickers:
    #ATR 3 Sell #1
    x = 0
    loop = iter(range(len(df)))
    for i in loop:
        df.loc[i, ('Stoploss3', t)] = df.loc[x:i, ('Stoploss_atr3', t)].max()
        try:
            if df.loc[i, ('Stoploss3', t)] == df.loc[i-4, ('Stoploss3', t)]:
                x=i
        except:
            pass
        if df.loc[i, ('Stoploss3', t)] >= df.loc[i, ('Close', t)]:
            df.loc[i:i+5, ('Strat_atr3_signal', t)] = 1
            next(islice(loop, 4, 4), None)
            x=i
        if df.loc[i, ('Stoploss3', t)] < df.loc[i, ('Close', t)]:
            df.loc[i, ('Strat_atr3_signal', t)] = 0
        else:
            df.loc[i, ('Strat_atr3_signal', t)] = 0

for t in tickers:
    #ATR 1 Sell #2
    x = 0
    loop = iter(range(len(df)))
    for i in loop:
        df.loc[i, ('Stoploss1', t)] = df.loc[x:i, ('Stoploss_atr1', t)].max()
        try:
            if df.loc[i, ('Stoploss1', t)] == df.loc[i-4, ('Stoploss1', t)]:
                x=i
        except:
            pass
        if df.loc[i, ('Stoploss1', t)] >= df.loc[i, ('Close', t)]:
            df.loc[i:i+5, ('Strat_atr1_signal', t)] = 1
            next(islice(loop, 4, 4), None)
            x=i
        if df.loc[i, ('Stoploss1', t)] < df.loc[i, ('Close', t)]:
            df.loc[i, ('Strat_atr1_signal', t)] = 0
        else:
            df.loc[i, ('Strat_atr1_signal', t)] = 0
            
for t in tickers:
    for i in range(1, len(df)-1):
    #Groter dan Upper
        if df.loc[i, ('Upper_bb', t)] < df.loc[i, ('Close', t)]:
            df.loc[i, ('Strat_upper_bb_signal', t)] = 1
        if df.loc[i, ('Upper_bb', t)] >= df.loc[i, ('Close', t)]:
            df.loc[i, ('Strat_upper_bb_signal', t)] = 0
    #Tussen de banden
    for i in range(1, len(df)-1):
        if df.loc[i, ('Upper_bb', t)] > df.loc[i, ('Close', t)] and df.loc[i, ('Lower_bb', t)] < df.loc[i, ('Close', t)]:
            df.loc[i, ('Strat_contained_bb_signal', t)] = 1
        if df.loc[i, ('Upper_bb', t)] < df.loc[i, ('Close', t)] and df.loc[i, ('Lower_bb', t)] > df.loc[i, ('Close', t)]:
            df.loc[i, ('Strat_contained_bb_signal', t)] = 0
            
for t in tickers:
    for i in range(1, len(df)-1):
    #Close 75
        if df.loc[i, ('Decycler_c75', t)] >= df.loc[i-1, ('Decycler_c75', t)]:
            df.loc[i, ('Strat_decycle_c75_signal', t)] = 1
        else:
            df.loc[i, ('Strat_decycle_c75_signal', t)] = 0
            
for t in tickers:
    for i in range(1, len(df)-1):
    #Open 25
        if df.loc[i, ('Decycler_o25', t)] >= df.loc[i-1, ('Decycler_o25', t)]: 
            df.loc[i, ('Strat_decycle_o25_signal', t)] = 1
        else:
            df.loc[i, ('Strat_decycle_o25_signal', t)] = 0
            
for t in tickers:
    for i in range(1, len(df)-1):
    #Open 75
        if df.loc[i, ('Decycler_o75', t)] >= df.loc[i-1, ('Decycler_o75', t)]: 
            df.loc[i, ('Strat_decycle_o75_signal', t)] = 1
        else:
            df.loc[i, ('Strat_decycle_o75_signal', t)] = 0    
            
for t in tickers:
    for i in range(1, len(df)-1):
        if df.loc[i, ('SMA10', t)] >= df.loc[i, ('SMA20', t)]:
            df.loc[i, ('Strat_sma_sma_signal', t)] = 1
        else:
            df.loc[i, ('Strat_sma_sma_signal', t)] = 0
    for i in range(1, len(df)-1):
        if df.loc[i, ('SMA10', t)] >= df.loc[i, ('Upper_bb', t)]:
            df.loc[i, ('Strat_sma_upper_signal', t)] = 1
        else:
            df.loc[i, ('Strat_sma_upper_signal', t)] = 0
    for i in range(1, len(df)-1):
        if df.loc[i, ('Upper_bb', t)] > df.loc[i, ('SMA10', t)] and df.loc[i, ('Lower_bb', t)] < df.loc[i, ('SMA10', t)]:
            df.loc[i, ('Strat_sma_bb_signal', t)] = 1
        if df.loc[i, ('Upper_bb', t)] < df.loc[i, ('SMA10', t)] and df.loc[i, ('Lower_bb', t)] > df.loc[i, ('SMA10', t)]:
            df.loc[i, ('Strat_sma_bb_signal', t)] = 0

time = datetime.datetime.now().time()
print(time)

In [39]:
#Sell Strat
for t in tickers:
    #df[('Strat_s', t)] = 0
    for i in range(len(df)-1):                    
        if df.loc[i, ('Strat_atr1_signal', t)] == 1:
            if df.loc[i, ('Strat_upper_bb_signal', t)] == 1:
                if df.loc[i, ('Strat_sma_upper_signal', t)] == 1:
                    df.loc[i, ('Strat_s', t)] = 1
        if df.loc[i, ('Strat_atr3_signal', t)] == 1:
            if df.loc[i, ('Strat_contained_bb_signal', t)] == 1:
                df.loc[i, ('Strat_s', t)] = 1
        else:
            df.loc[i, ('Strat_s', t)] = 0

#Buy strat  
for t in tickers:
    #df[('Strat_b', t)] = 1
    for i in range(len(df)-1):
        if df.loc[i, ('Strat_s', t)] == 1 and df.loc[i, ('Strat_upper_bb_signal', t)] == 1:
            if df.loc[i-1, ('Strat_s', t)] == 1 and df.loc[i-1, ('Strat_upper_bb_signal', t)] == 1:
                if df.loc[i-2, ('Strat_s', t)] == 1 and df.loc[i-2, ('Strat_upper_bb_signal', t)] == 1:
                    df.loc[i, ('Strat_b', t)] = 1                        
        #if df.loc[i, ('Strat_decycle_o25_signal', t)] == 1:
         #   if df.loc[i, ('Strat_sma_sma_signal', t)] == 1:
          #      df.loc[i, ('Strat_b', t)] = 
        if df.loc[i, ('Strat_decycle_o75_signal', t)] == 1:
            if df.loc[i, ('Strat_sma_bb_signal', t)] == 1:
                df.loc[i, ('Strat_b', t)] = 1
        else:
            df.loc[i, ('Strat_b', t)] = 0

#Bringing it together
for t in tickers:
    i=0
    df.loc[:75, ('Strat', t)]=1 #Omdat langste SMA = 75
    while i < len(df)-1: 
        if df.loc[i, ('Strat_s', t)] == 1:
            while True:
                if df.loc[i, ('Strat_b', t)] == 0:
                    df.loc[i+1, ('Strat', t)] = 0
                    i+=1
                if df.loc[i, ('Strat_b', t)] == 1:
                    df.loc[i+1, ('Strat', t)] = 1
                    i+=1
                    break               
        if df.loc[i, ('Strat_s', t)] == 0:
            df.loc[i+1, ('Strat', t)] = 1
            i+=1

df[('Strat_return', 'Total')] = 0
for t in tickers:
    df[('Percent_return', t)] = df[('Close', t)].pct_change()
    df[('Percent_return', 'Total')] = df[('Close', t)].pct_change()
    
    df[('Strat_return', t)] = df[('Strat', t)] * df[('Percent_return', t)]
    df[('Strat_return', 'Total')] += df[('Strat_return', t)].cumsum()
    
time = datetime.datetime.now().time()
print(time)

03:40:06.475362


In [40]:
for t in tickers:
    fig, ax = plt.subplots(5, sharex=True, figsize=(10,5), gridspec_kw={'height_ratios':[2.5, 0.5, 1.5, 0.5, 0.5], 'hspace':0.02})
    ax[0].set_title(t)
    ax[0].plot(df[('Close', t)])
    ax[1].plot(df[('Strat', t)])
    ax[2].plot(df[('Percent_return', 'Total')].cumsum(), linewidth=1)
    ax[2].plot(df[('Strat_return', t)].cumsum(), linewidth=1, color='r')
    ax[3].plot(df[('Strat_b', t)])
    ax[4].plot(df[('Strat_s', t)])
    r1=df[('Strat_return', t)].sum()
    r2=df[('Percent_return', t)].sum()
    print(f'Strat return SPY = {r1}')
    print(f'Hold return SPY = {r2}')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Strat return SPY = 0.31539179428341224
Hold return SPY = 0.5041826175961925


In [ ]:
print(df.index.max())
with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
    display(df)